In [4]:
"""
{
  [assessment_name]: {
    "description": [description text],
    "url": [reproschema_url],
    "data elements": {
       [element_name]: {
          'description': [description text],
          'datatype': <type>,
          'choices': [
             [choice name],
             [another choice name],
           ],
           'termURL': [reproschema_url]
       }
     } 
   }
}
"""

"""
Ex now:
{
    "adhd_session_id": {
        "description": "Unique identifier for the ADHD session."
    }
}
"""
#dict of non-items only that match
#list of keys that don't correspond to any reproschema files
#
# def
# for each phenotype jsons
#   open as dict
#   for each key
#       recursively search reproschema for item files that match
#       if found at least one:
#           if at least one type item
#               then populate data
#           else
#              add to list of non-items only that match 
#       else:
#           add to list of keys that don't correspond to any reproschema files
#
# def populate data
#           

'\nEx:\n{\n    "adhd_session_id": {\n        "description": "Unique identifier for the ADHD session."\n    }\n'

In [16]:
import os
import json

def search_string_in_json_files(directory, search_string):
    matching_files = []  # List to store paths of matching JSON files

    # Walk through each directory and file in the given directory
    for root, dirs, files in os.walk(directory):
        for phenotype_file_name in files:
            file_path = os.path.join(root, phenotype_file_name)
            try:
                # Attempt to open and load the file as JSON
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)  # Load JSON data
                    # Check if the search string is present in the JSON content
                    if search_string in str(data):
                        matching_files.append(file_path)
            except Exception:
                # Skip files that cannot be read or loaded as JSON
                continue
    
    return matching_files  # Return the list of matching file paths
# search_string = "adhd_session_id"
# directory = "/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities/q_mood_adhd_adult/items"
# search_string_in_json_files(directory, search_string)


In [ ]:
def populate_data_entry(phenotype_file_dict, key, item_file_path):
    pass

In [18]:
matching_non_item_files = {}
multiple_item_files = []
non_matching = []

# Specify the phenotype_dir containing the .json files
phenotype_dir = "/Users/isaacbevers/sensein/b2ai-wrapper/b2aiprep/src/b2aiprep/prepare/resources/b2ai-data-bids-like-template/phenotype"
b2ai_redcap2rs_activities_dir = "/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities"
# Dictionary to store the loaded JSON data

def main():
    # Loop through each file in the phenotype_dir
    for phenotype_file_name in os.listdir(phenotype_dir):
        # Check if the file ends with .json and is not "<measurement_tool_name>.json"
        if phenotype_file_name.endswith(".json") and phenotype_file_name != "<measurement_tool_name>.json":
            file_path = os.path.join(phenotype_dir, phenotype_file_name)

            # Open and load the JSON file
            with open(file_path, 'r', encoding='utf-8') as file:
                phenotype_file_dict = json.load(file)

            for key in phenotype_file_dict:
                file_paths = search_string_in_json_files(b2ai_redcap2rs_activities_dir, key)
                if file_paths:
                    item_file_paths = [path for path in file_paths if "item" in path]
                    if item_file_paths and len(item_file_paths) == 1:
                        populate_data_entry(phenotype_file_dict, key, item_file_paths[0])
                    elif item_file_paths and len(item_file_paths) > 1:
                        multiple_item_files.append(item_file_paths)
                    else:
                        matching_non_item_files[key] = file_paths
                else:
                    non_matching.append(key)
            # save data

#       if found at least one:
#           if at least one type item
#               then populate data
#           else
#              add to list of non-items only that match 
#       else:
#           add to list of keys that don't correspond to any reproschema files
        
        
